In [18]:
import numpy as np

BigX = np.load('data/soybean_data_compressed.npz') ## order: locID, year, yield, W(52*6), S(6*11), P(14)
X=BigX['data']

np.set_printoptions(suppress=True, precision=2)
print(X[0:3, 0:5])

[[   0.   1980.     32.5     0.27    0.  ]
 [   0.   1981.     36.      0.6     0.  ]
 [   0.   1982.     37.      2.1     0.38]]


In [19]:
X_train = X[X[:,1]<=2017]
print(X_train[0:3, 0:5])

X_train = X_train[:,3:]
print(X_train[0:3, 0:5])

[[   0.   1980.     32.5     0.27    0.  ]
 [   0.   1981.     36.      0.6     0.  ]
 [   0.   1982.     37.      2.1     0.38]]
[[0.27 0.   1.62 0.4  0.97]
 [0.6  0.   0.04 0.   0.86]
 [2.1  0.38 1.68 0.53 6.34]]


In [20]:
M=np.mean(X_train, axis=0, keepdims=True)
S=np.std(X_train, axis=0, keepdims=True)
X[:,3:] = (X[:,3:]-M)/S

/tmp/ipykernel_7732/176784504.py:3: RuntimeWarning: invalid value encountered in divide
  X[:,3:] = (X[:,3:]-M)/S


In [21]:
# Remove low yield observations
X=np.nan_to_num(X)
index_low_yield=X[:,2]<5
print('low yield observations',np.sum(index_low_yield))
print(X[index_low_yield][:,1])
X=X[np.logical_not(index_low_yield)]
del BigX

low yield observations 2
[1988. 2003.]


In [22]:
Index=X[:,1]==2018

print('Std %.2f and mean %.2f  of test ' %(np.std(X[Index][:,2]), np.mean(X[Index][:,2])))
print("train data",np.sum(np.logical_not(Index)))
print("test data",np.sum(Index))

Std 10.55 and mean 53.94  of test 
train data 24871
test data 472


In [23]:
print(X.shape)

(25343, 395)


In [42]:
years = np.arange(1980, 2019)
avg_values = {str(year): np.mean(X[X[:, 1] == year][:, 2]) for year in years}
print(avg_values) # --> AVERAGE YIELD FOR EACH YEAR (INCLUDING 2018)

{'1980': np.float64(28.896072507552866), '1981': np.float64(33.37415565345081), '1982': np.float64(32.86419019316493), '1983': np.float64(26.874306569343062), '1984': np.float64(27.47536231884058), '1985': np.float64(34.75277777777778), '1986': np.float64(35.56026200873362), '1987': np.float64(35.70191176470588), '1988': np.float64(26.670444763271156), '1989': np.float64(32.388294797687855), '1990': np.float64(34.28272327964861), '1991': np.float64(34.376106194690266), '1992': np.float64(37.20850439882698), '1993': np.float64(33.805044510385756), '1994': np.float64(41.85523952095809), '1995': np.float64(35.0006015037594), '1996': np.float64(38.595927601809954), '1997': np.float64(39.79051094890511), '1998': np.float64(39.97238372093023), '1999': np.float64(37.414410480349346), '2000': np.float64(36.899146514935985), '2001': np.float64(39.67424023154848), '2002': np.float64(37.19884393063584), '2003': np.float64(32.673932253313694), '2004': np.float64(42.45685131195336), '2005': np.floa

In [43]:
avg2 = list(avg_values.values())
mm = np.mean(avg2)
ss = np.std(avg2)

avg = {str(year): (value - mm) / ss for year, value in avg_values.items()}
avg['2018'] = avg['2017']
print(avg) # --> NORMALIZED AVERAGE YIELD FOR EACH YEAR (2018 REPLACED BY 2017)

{'1980': np.float64(-1.4582864093819747), '1981': np.float64(-0.8110332591297735), '1982': np.float64(-0.8847426432425297), '1983': np.float64(-1.7505084003420137), '1984': np.float64(-1.6636330086477271), '1985': np.float64(-0.6117699842181856), '1986': np.float64(-0.49505783412246773), '1987': np.float64(-0.4745840626898213), '1988': np.float64(-1.7799741766344916), '1989': np.float64(-0.9535276085675289), '1990': np.float64(-0.6797107179633866), '1991': np.float64(-0.6662133388457581), '1992': np.float64(-0.2568241871334897), '1993': np.float64(-0.7487534485560939), '1994': np.float64(0.4148055812735497), '1995': np.float64(-0.5759500402549351), '1996': np.float64(-0.05628882197700863), '1997': np.float64(0.11637385744519654), '1998': np.float64(0.142661382889246), '1999': np.float64(-0.22706293537032865), '2000': np.float64(-0.30153815477994156), '2001': np.float64(0.09956832131751261), '2002': np.float64(-0.2582204918169693), '2003': np.float64(-0.9122421440148659), '2004': np.flo

In [ ]:
# add normalized average yield to the data based on year
X = np.concatenate((X, np.array([avg[str(int(year))] for year in X[:, 1]]).reshape(-1, 1)), axis=1)
print(X.shape)


(25343, 396)


In [ ]:
print(np.mean(X[X[:, 1] == 2015][:, -1]))
print(np.mean(X[X[:, 1] == 2016][:, -1]))
print(np.mean(X[X[:, 1] == 2017][:, -1]))
print(np.mean(X[X[:, 1] == 2018][:, -1]))

1.6528674736360225
2.2986134626530217
1.75394127659999
1.75394127659999


In [98]:
batch_size = 3
time_steps = 2

In [ ]:
out = np.zeros(shape=[batch_size, time_steps, 396])

In [113]:
for i in range(batch_size):
    r1 = np.random.randint(0, 35)
    years = np.array([(r1 + i) for i in range(time_steps)]) + 1980
    print(years)
    
    for j, y in enumerate(years):
        r2 = np.random.randint(X[X[:, 1] == y].shape[0])
        out[i, j, :] = X[X[:, 1] == y][r2, :]

print(out)
print(out.shape)

[1997 1998]
[2001 2002]
[2001 2002]
[[[ 363.   1997.     37.   ...   -0.07   -0.06    0.12]
  [  30.   1998.     46.   ...   -0.07   -0.06    0.14]]

 [[ 575.   2001.     38.   ...   -0.07   -0.06    0.1 ]
  [ 653.   2002.     36.6  ...   -0.07   -0.06   -0.26]]

 [[ 736.   2001.     32.2  ...   -0.07   -0.06    0.1 ]
  [1029.   2002.     38.3  ...   -0.07   -0.06   -0.26]]]
(3, 2, 396)


In [351]:
BigX = np.load('data/soybean_data_compressed.npz') ## order: locID, year, yield, W(52*6), S(6*11), P(14)
X=BigX['data']

np.set_printoptions(suppress=True, precision=2)
print(X[0:3, 0:5])

[[   0.   1980.     32.5     0.27    0.  ]
 [   0.   1981.     36.      0.6     0.  ]
 [   0.   1982.     37.      2.1     0.38]]


In [352]:
def preprocess_data(X):
    
    # 1. remove low yield observations
    X = np.nan_to_num(X)
    index_low_yield = X[:,2] < 5
    print('low yield observations', np.sum(index_low_yield))
    print(X[index_low_yield][:, 1])
    X = X[np.logical_not(index_low_yield)]
    
    # 2. calculate and append average yield of each year
    years = np.arange(1980, 2019)
    avg = {str(year): np.mean(X[X[:, 1] == year][:, 2]) for year in years}
    avg['2018'] = avg['2017']
    X = np.concatenate((X, np.array([avg[str(int(year))] for year in X[:, 1]]).reshape(-1, 1)), axis=1)
    
    # 3. standardize the data on the training data only
    X_train = X[X[:,1] <= 2017][:, 3:]

    M=np.mean(X_train, axis=0, keepdims=True)
    S=np.std(X_train, axis=0, keepdims=True)
    epsilon = 1e-8
    
    X[:,3:] = (X[:,3:] - M) / (S + epsilon)
    
    print(X[0:3, 0:5])
    return X

In [353]:
X = preprocess_data(X)

batch_size = 30
time_steps = 5

low yield observations 2
[1988. 2003.]
[[   0.   1980.     32.5    -0.42   -0.57]
 [   0.   1981.     36.     -0.28   -0.57]
 [   0.   1982.     37.      0.37   -0.36]]


In [354]:
def get_sample(X, batch_size, time_steps):
    sample = np.zeros(shape = [batch_size, time_steps, X.shape[1] + time_steps])

    for i in range(batch_size):
        r1 = np.random.randint(1979 + time_steps, 2019)            # random start year for each batch
        years = np.array([(r1 + i - time_steps + 1) for i in range(time_steps)])
        
        for j, y in enumerate(years):
            r2 = np.random.randint(0, X[X[:, 1] == y].shape[0])    # random observation for the specific year         
            obs = X[X[:, 1] == y][r2, :]
            
            avg_yield_values = []
            for k in range(time_steps):
                prev_y = y - k - 1
                prev_data = X[X[:, 1] == prev_y]
                
                if prev_y < 1980:
                    avg_yield_values.append(0)
                else:
                    avg_yield_values.append(prev_data[r2, -1])
            
            avg_yield_values = np.array(avg_yield_values[::-1])
            obs_with_avg_yield = np.concatenate((obs, avg_yield_values))
            sample[i, j, :] = obs_with_avg_yield

    return sample.reshape(-1, X.shape[1] + time_steps) 

In [355]:
get_sample(X, batch_size, time_steps)

IndexError: index 679 is out of bounds for axis 0 with size 663

In [ ]:
def get_sample_test(X, time_steps):
    sample = []
    X_test = X[X[:, 1] == 2018]

    for obs in X_test:
        avg_yield_values = []
        
        for k in range(time_steps):
            prev_y = 2018 - k - 1
            prev_data = X[X[:, 1] == prev_y]
            avg_yield_values.append(prev_data[0, -1])
        
        avg_yield_values = np.array(avg_yield_values[::-1])
        obs_with_avg_yield = np.concatenate((obs, avg_yield_values))
        sample.append(obs_with_avg_yield)

    return np.array(sample)   

In [350]:
get_sample_test(X, time_steps)


[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.0193111075116974)]
[np.float64(2.0193111075116974), np.float64(2.6095807307270666)]
[np.float64(2.019311

array([[   0.  , 2018.  ,   60.7 , ...,    2.02,    2.61,    2.02],
       [   3.  , 2018.  ,   61.8 , ...,    2.02,    2.61,    2.02],
       [   4.  , 2018.  ,   61.7 , ...,    2.02,    2.61,    2.02],
       ...,
       [1041.  , 2018.  ,   50.9 , ...,    2.02,    2.61,    2.02],
       [1042.  , 2018.  ,   52.  , ...,    2.02,    2.61,    2.02],
       [1043.  , 2018.  ,   39.4 , ...,    2.02,    2.61,    2.02]])

In [276]:
def get_sample(batch_size, time_steps):
    sample = np.zeros(shape = [batch_size, time_steps, 396])
    
    for i in range(batch_size):
        r1 = np.random.randint(1979 + time_steps, 2019)            # random start year for each batch
        years = np.array([(r1 + i - time_steps + 1) for i in range(time_steps)])
        
        for j, y in enumerate(years):
            r2 = np.random.randint( X[X[:, 1] == y].shape[0])    # random observation for each time step         
            sample[i, j, :] = X[X[:, 1] == y][r2, :]
            
    return sample.reshape(-1, 3+6*52+66+14+1)                    # shape (batch_size=25, time_steps=5, features=396)

In [277]:
X_test = X[X[:,1] == 2018][:, 3:]
y_test = X[X[:,1] == 2018][:, 2]

print(X.shape)
X = get_sample(batch_size, time_steps) if batch_size > 0 else X
print(X.shape)
X_train = X[X[:,1] <= 2017][:, 3:]
y_train = X[X[:,1] <= 2017][:, 2]

print('Std %.2f and mean %.2f  of test ' %(np.std(y_test[:]), np.mean(y_test[:])))
print("train data", X_train.shape)
print("test data", X_test.shape)

(25343, 396)
(6, 396)
Std 10.55 and mean 53.94  of test 
train data (6, 393)
test data (472, 393)
